In [7]:
import pandas as pd
import numpy as np
import math
#Insert your features and labels
samples = np.load(r'D:\Downloads Folder\activation_maps_lines_image52_pixel.npy')
features = np.load(r'D:\Downloads Folder\activation_maps_lines_image52_labelpixel.npy')

In [8]:
#Creating a table
table = {
    'age': ['<=30', '<=30', '31...40', '>40', '>40', '>40', '31...40', '<=30', '<=30', '>40', '<=30', '31...40', '31...40', '>40'],
    'income': ['high', 'high', 'high', 'medium', 'low', 'low', 'low', 'medium', 'low', 'medium', 'medium', 'medium', 'high', 'medium'],
    'student': ['no', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'no'],
    'credit_rating': ['fair', 'excellent', 'fair', 'fair', 'fair', 'excellent', 'excellent', 'fair', 'fair', 'fair', 'excellent', 'excellent', 'fair', 'excellent'],
    'buys_computer': ['no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no']
}

dataframe = pd.DataFrame(table, index= [0,1,2,3,4,5,6,7,8,9,10,11,12,13])
print(dataframe)

        age  income student credit_rating buys_computer
0      <=30    high      no          fair            no
1      <=30    high      no     excellent            no
2   31...40    high      no          fair           yes
3       >40  medium      no          fair           yes
4       >40     low     yes          fair           yes
5       >40     low     yes     excellent            no
6   31...40     low     yes     excellent           yes
7      <=30  medium      no          fair            no
8      <=30     low     yes          fair           yes
9       >40  medium     yes          fair           yes
10     <=30  medium     yes     excellent           yes
11  31...40  medium      no     excellent           yes
12  31...40    high     yes          fair           yes
13      >40  medium      no     excellent            no


In [9]:
# Question A1:
#Functions of the entropy before,after and Information gain for each attribute at root node
def entropyBefore(data_field,attribute_value):
    positive_data_field = data_field.value_counts()[attribute_value]
    total_data_field = data_field.count()
    negative_data_field = total_data_field - positive_data_field
    positive_probability = positive_data_field / total_data_field
    negative_probability = negative_data_field / total_data_field
    return -(positive_probability * float(np.log2(positive_probability)) + negative_probability * float(np.log2(negative_probability)))

def entropyAfter(data_field,attribute_value):
    summation = data_field.value_counts()[attribute_value] / data_field.count()
    return summation * entropyBefore(data_field,attribute_value)

def Informationgain(data_field,attribute_value):
    return entropyBefore(data_field,attribute_value) - entropyAfter(data_field,attribute_value)
            


In [10]:
# Age attributes
attribute_of_age = dataframe.iloc[:,0]
print("The list of attributes in age are:\n ",attribute_of_age)

The list of attributes in age are:
  0        <=30
1        <=30
2     31...40
3         >40
4         >40
5         >40
6     31...40
7        <=30
8        <=30
9         >40
10       <=30
11    31...40
12    31...40
13        >40
Name: age, dtype: object


In [11]:
entropy_before_age_lessthan_equal_30 = entropyBefore(attribute_of_age,'<=30') #Finding the entropy before for <=30
entropy_before_age_30_to_40_range = entropyBefore(attribute_of_age,'31...40')  #Finding the entropy before for 31...40
entropy_before_age_greater_40 = entropyBefore(attribute_of_age,'>40')         #Finding the entropy before for >40

print("The before entropy of the attribute age,when less than equal to 30:",entropy_before_age_lessthan_equal_30)
print("The before entropy of the attribute age of range of 31 ro 40:",entropy_before_age_30_to_40_range)
print("The before entropy of the attribute age,when greater than 40: ",entropy_before_age_greater_40)


The before entropy of the attribute age,when less than equal to 30: 0.9402859586706311
The before entropy of the attribute age of range of 31 ro 40: 0.863120568566631
The before entropy of the attribute age,when greater than 40:  0.9402859586706311


In [12]:
entropy_after_age_lessthan_equal_30 = entropyAfter(attribute_of_age,'<=30')   #Finding the entropy after for <=30
entropy_after_age_30_to_40_range = entropyAfter(attribute_of_age,'31...40')   #Finding the entropy after for 31...40
entropy_after_age_greater_40 = entropyAfter(attribute_of_age,'>40')          #Finding the entropy after for >40

print("The after entropy of the attribute age,when less than equal to 30:",entropy_after_age_lessthan_equal_30)
print("The after entropy of the attribute age of range of 31 ro 40:",entropy_after_age_30_to_40_range)
print("The after entropy of the attribute age,when greater than 40: ",entropy_after_age_greater_40)


The after entropy of the attribute age,when less than equal to 30: 0.3358164138109397
The after entropy of the attribute age of range of 31 ro 40: 0.24660587673332313
The after entropy of the attribute age,when greater than 40:  0.3358164138109397


In [13]:
Information_gain_lessthan_30 = Informationgain(attribute_of_age,'<=30')       #Information gain for <=30
Information_gain_30_to_40_range = Informationgain(attribute_of_age,'31...40') #Information gain for 31...40
Information_gain_greater_40 = Informationgain(attribute_of_age,'>40')         #Information gain for >40

print("The information gain of attribute age,when less than equal to 30:",Information_gain_lessthan_30)
print("The information gain of attribute age,of range of 31 ro 40:",Information_gain_30_to_40_range)
print("The information gain of attribute age,when greater than 40:",Information_gain_greater_40)

The information gain of attribute age,when less than equal to 30: 0.6044695448596915
The information gain of attribute age,of range of 31 ro 40: 0.6165146918333079
The information gain of attribute age,when greater than 40: 0.6044695448596915
